### Step 1: Loading the Data

First, let's load the dataset into a pandas DataFrame.

###### Step 1.1: Get the file location


In [1]:
## file folder
import os

folder_path = '/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs'

# Get the list of files and directories in the folder
folder_content = os.listdir(folder_path)

# Print the content of the folder
for item in folder_content:
    print(item)

CTRL_1
CTRL_2
CTRL_2D
GEM_2
TGFb1_1
TGFb1_2
TGFb1_GEM_1
TGFb1_GEM_2
combined_adata.h5ad
filtered_preprocessed_data.h5ad
filtered_preprocessed_combined_adata.h5ad
annotated_adata.h5ad
hvg_adata.h5ad
clustered_adata.h5ad


In [2]:
## print head of example file
import csv

def print_csv_head(file_path, num_rows):
    with open(file_path, 'r') as file:
        csv_reader = csv.reader(file)
        header = next(csv_reader)  # Read the header
        print(','.join(header))  # Print the header
        
        # Print the specified number of rows
        for _ in range(num_rows):
            row = next(csv_reader, None)
            if row is None:
                break
            print(','.join(row))

file_path = '/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_1/metrics_summary.csv'
num_rows = 5  # Specify the number of rows to print

print_csv_head(file_path, num_rows)

Category,Library Type,Grouped By,Group Name,Metric Name,Metric Value
Cells,Gene Expression,,,Cells,2,973
Cells,Gene Expression,,,Confidently mapped antisense,3.31%
Cells,Gene Expression,,,Confidently mapped to exonic regions,86.53%
Cells,Gene Expression,,,Confidently mapped to genome,96.22%
Cells,Gene Expression,,,Confidently mapped to intergenic regions,2.59%


In [7]:
# Load the gene expression matrices for each sample
print(folder_content)


['CTRL_1', 'CTRL_2', 'CTRL_2D', 'GEM_2', 'TGFb1_1', 'TGFb1_2', 'TGFb1_GEM_1', 'TGFb1_GEM_2', 'combined_adata.h5ad', 'filtered_preprocessed_data.h5ad', 'filtered_preprocessed_combined_adata.h5ad', 'annotated_adata.h5ad', 'hvg_adata.h5ad', 'clustered_adata.h5ad']


##### Step 1.2: Unzip the data


In [4]:
import os
import gzip


In [8]:
def unzip_all_gz_files(folder_path):
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        

        if file_name.endswith('.gz'):
            unzipped_file_path = os.path.splitext(file_path)[0]  # Remove the .gz extension
            #print(file_path)
            with gzip.open(file_path, 'rb') as gz_file:
                with open(unzipped_file_path, 'wb') as unzipped_file:
                    unzipped_file.write(gz_file.read())
            
            # Optionally, you can delete the .gz file after unzipping
            # os.remove(file_path)


samples = folder_content
# or
samples = ['CTRL_1', 'CTRL_2', 'CTRL_2D', 'GEM_2', 'TGFb1_1', 'TGFb1_2', 'TGFb1_GEM_1', 'TGFb1_GEM_2']
for sample in samples:

    folder_path = f'/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/{sample}/count/sample_filtered_feature_bc_matrix'
    df = unzip_all_gz_files(folder_path)

    #print(df.head())
    print(folder_path)
    

/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_1/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_2/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/CTRL_2D/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/GEM_2/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_1/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_2/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_GEM_1/count/sample_filtered_feature_bc_matrix
/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/TGFb1_GEM_2/count/sample_filtered_feature_bc_matrix


###### Step 1.3: Load the files
example

https://support.10xgenomics.com/single-cell-gene-expression/software/pipelines/latest/output/matrices

In [9]:
from tqdm import tqdm
import pandas as pd

In [10]:
import scipy.io as sio
import os

In [11]:
import anndata as ad

In [12]:
import scanpy as sc

In [13]:
!conda list

/bin/bash: line 1: conda: command not found


In [14]:
#adata = ad.AnnData()
adata_list = []

In [15]:
combined_adata = ad.AnnData()

In [17]:
#samples = ['sample1', 'sample2']  # Add more sample names if needed
conditions = folder_content
conditions = samples
#pwd /storage/users/sac43cg/res_Samantha/outs/per_sample_outs/CTRL_1/count/sample_filtered_feature_bc_matrix

# Create an empty list to store the AnnData objects
adata_list = []
barcodes_list = []
features_list = []
matrix_list = []
#combined_adata = []

for condition in conditions:
           
    # Define the file paths
    base_path = f'/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/{condition}/count/sample_filtered_feature_bc_matrix/'  # Replace with the actual path to your data

    barcodes_file = os.path.join(base_path, 'barcodes.tsv')
    features_file = os.path.join(base_path, 'features.tsv')
    matrix_file = os.path.join(base_path, 'matrix.mtx')

    # Read the barcodes.tsv file
    barcodes = pd.read_csv(barcodes_file, header=None, index_col=0, names=['barcode'])

    # Read the features.tsv file
    features = pd.read_csv(features_file, sep='\t', header=None, index_col=0, names=['feature_id', 'symbol', 'type'])
    # get only gene expression features
    #features = features[features['type'] == 'gene expression']

    # Read the matrix.mtx file
    matrix = sio.mmread(matrix_file)

    # Create the AnnData object
    adata = ad.AnnData(X=matrix.T, obs=barcodes, var=features)
    
    # Add a column to the observation metadata to indicate the condition
    adata.obs['condition'] = condition
    #adata.varm['ensembl'] = features['feature_id']
    #adata.varp['symbol'] = features['symbol']
    #adata.varp['type'] = features['type']
    #adata.varm['condition'] = condition

    # Optional: If you have additional annotations for cells or features, you can add them to the AnnData object as well.
    # For example, if you have a file called 'annotations.csv' with additional cell annotations:
    #annotations_file = os.path.join(base_path, 'annotations.csv')
    #annotations = pd.read_csv(annotations_file, index_col=0)
    # Add cell annotations
    #adata.obs = adata.obs.merge(annotations, left_index=True, right_index=True)

    # Append the AnnData object to the list
    adata_list.append(adata)
    features['condition'] = condition
    features_list.append(features)
        
    
# Combine all the AnnData objects into a single AnnData object
combined_adata = ad.concat(adata_list, join='outer')

# Rename duplicate variable and observation names
#combined_adata.var_names_make_unique()
#combined_adata.obs_names_make_unique()




In [18]:
combined_adata.write('/storage/users/sac43cg/res_Samantha_1/outs/per_sample_outs/combined_adata.h5ad')

In [19]:
#Step 1.3.1: Check the new adata (.h5ad) file


In [20]:
# Print the annotations
print("Observation annotations:")
print(combined_adata.obs.head())  # Print the first few rows of observation annotations

print("\nVariable annotations:")
print(combined_adata.var.head())  # Print the first few rows of variable annotations

print("\nVariable annotations:")
print(combined_adata.varm)  # Print the first few rows of variable annotations

print("\nVariable features:")
print(features.head())
print(features['type'].unique())

# Print the dimensions
print("\nNumber of observations (cells):", combined_adata.n_obs)
print("Number of variables (features):", combined_adata.n_vars)



Observation annotations:
                   condition
barcode                     
AAACCCAGTAGGCTCC-1    CTRL_1
AAACCCAGTCGCAACC-1    CTRL_1
AAACCCATCCACAAGT-1    CTRL_1
AAACGCTTCACTACGA-1    CTRL_1
AAACGCTTCAGCCTCT-1    CTRL_1

Variable annotations:
Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945]

Variable annotations:
AxisArrays with keys: 

Variable features:
                      symbol             type    condition
feature_id                                                
ENSG00000243485  MIR1302-2HG  Gene Expression  TGFb1_GEM_2
ENSG00000237613      FAM138A  Gene Expression  TGFb1_GEM_2
ENSG00000186092        OR4F5  Gene Expression  TGFb1_GEM_2
ENSG00000238009   AL627309.1  Gene Expression  TGFb1_GEM_2
ENSG00000239945   AL627309.3  Gene Expression  TGFb1_GEM_2
['Gene Expression' 'Multiplexing Capture']

Number of observations (cells): 11919
Number of variables (features): 36613


In [29]:
# Iterate over each condition
for condition in combined_adata.obs['condition'].unique():
    condition_adata = combined_adata[combined_adata.obs['condition'] == condition]
    
    # Get the number of features and barcodes for the current condition
    num_features = condition_adata.n_vars
    num_barcodes = condition_adata.n_obs
    
    print("Observation annotations:")
    print(condition_adata.obs.head())  # Print the first few rows of observation annotations
    
    print("\nVariable annotations:")
    print(condition_adata.var.head())  # Print the first few rows of variable annotations
    
    # Write the condition name, number of features, and number of barcodes to the file
    print(f"Condition: {condition}\n")
    print(f"Number of features: {num_features}\n")
    print(f"Number of barcodes: {num_barcodes}\n\n")

# Close the file

Observation annotations:
                   condition
barcode                     
AAACCCAGTAGGCTCC-1    CTRL_1
AAACCCAGTCGCAACC-1    CTRL_1
AAACCCATCCACAAGT-1    CTRL_1
AAACGCTTCACTACGA-1    CTRL_1
AAACGCTTCAGCCTCT-1    CTRL_1

Variable annotations:
Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945]
Condition: CTRL_1

Number of features: 36613

Number of barcodes: 2973


Observation annotations:
                   condition
barcode                     
AAAGGATGTAGGAGTC-1    CTRL_2
AAAGGGCCACATACGT-1    CTRL_2
AAAGGGCGTTGAGAGC-1    CTRL_2
AAAGGTACAGAGCTAG-1    CTRL_2
AAAGGTAGTAGAGACC-1    CTRL_2

Variable annotations:
Empty DataFrame
Columns: []
Index: [ENSG00000243485, ENSG00000237613, ENSG00000186092, ENSG00000238009, ENSG00000239945]
Condition: CTRL_2

Number of features: 36613

Number of barcodes: 450


Observation annotations:
                   condition
barcode                     
AAACCCAAGAGGTATT-1   CTRL_2D

In [30]:
print(type(barcodes))
print(type(features))
print(type(adata))
print('\n')


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'anndata._core.anndata.AnnData'>




In [31]:
dimensions = barcodes.shape
print('\n' , dimensions)
dimensions = features.shape
print('\n' , dimensions)
dimensions = combined_adata.shape
print('\n' , dimensions)
dimensions = matrix.shape
print('\n' , dimensions)


 (485, 0)

 (36613, 3)

 (11919, 36613)

 (36613, 485)


### end of loading